Import thư viện

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Kích thước ảnh và batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

# Đường dẫn đến thư mục chứa dữ liệu
data_dir = '/content/drive/MyDrive/ISIC_2018/'
images_dir = os.path.join(data_dir, 'ISIC2018_Task3_Training_Input/ISIC2018_Task3_Training_Input')
labels_path = os.path.join(data_dir, 'ISIC2018_Task3_Training_GroundTruth.csv')


Load và chia dữ liệu

In [ ]:
# Đọc file labels.csv
df = pd.read_csv(labels_path)

# Xác định cột nhãn (label) từ các cột chẩn đoán
diagnosis_columns = df.columns[1:]  # Bỏ cột đầu tiên là 'image'

# Tạo cột 'label' chứa tên loại bệnh tương ứng
df['label'] = df[diagnosis_columns].idxmax(axis=1)

# Chia dữ liệu thành train (75%) và temp (25%)
from sklearn.model_selection import train_test_split
train_df, temp_df = train_test_split(df, test_size=0.25, stratify=df['label'], random_state=42)

# Chia temp thành validation (15%) và test (10%)
val_df, test_df = train_test_split(temp_df, test_size=0.4, stratify=temp_df['label'], random_state=42)


In [ ]:
total_images = df.shape[0]
print(f"Tổng số ảnh: {total_images}")


Tổng số ảnh: 10015


In [ ]:
train_df['image'] = train_df['image'].apply(lambda x: x if x.endswith('.jpg') else x + '.jpg')
val_df['image'] = val_df['image'].apply(lambda x: x if x.endswith('.jpg') else x + '.jpg')
test_df['image'] = test_df['image'].apply(lambda x: x if x.endswith('.jpg') else x + '.jpg')


In [ ]:
print(f"Tổng số ảnh trong tập huấn luyện (train): {len(train_df)}")
print(f"Tổng số ảnh trong tập xác thực (validation): {len(val_df)}")
print(f"Tổng số ảnh trong tập kiểm tra (test): {len(test_df)}")


Tổng số ảnh trong tập huấn luyện (train): 7511
Tổng số ảnh trong tập xác thực (validation): 1502
Tổng số ảnh trong tập kiểm tra (test): 1002


Xử lý mất cân bằng với weight

In [ ]:
# Tính toán class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['label']),
    y=train_df['label']
)

# Chuyển class_weights thành dictionary
class_weights_dict = dict(zip(np.unique(train_df['label']), class_weights))


Tăng cường dữ liệu

In [ ]:
# Tăng cường dữ liệu cho tập huấn luyện
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Chỉ rescale cho tập validation và test
val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Tạo generator cho từng tập
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=images_dir,
    x_col='image',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_dataframe(
    val_df,
    directory=images_dir,
    x_col='image',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_dataframe(
    test_df,
    directory=images_dir,
    x_col='image',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


Found 7511 validated image filenames belonging to 7 classes.
Found 1502 validated image filenames belonging to 7 classes.
Found 1002 validated image filenames belonging to 7 classes.


Xây dựng mô hình fine-tune RestNet50

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D

# Số lượng lớp (số loại bệnh)
num_classes = df['label'].nunique()

# Tải mô hình ResNet50 với trọng số từ ImageNet, không bao gồm các lớp fully connected ở trên cùng
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

# Đóng băng các lớp convolutional
for layer in base_model.layers:
    layer.trainable = False

# Thêm các lớp fully connected tùy chỉnh
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Tạo mô hình hoàn chỉnh
model = Model(inputs=base_model.input, outputs=predictions)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Huấn luyện và lưu checkpoint vào drive

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam

# Biên dịch mô hình
optimizer = Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Mount Google Drive (nếu chưa mount)
from google.colab import drive
drive.mount('/content/drive')

# Tạo thư mục nếu chưa tồn tại
import os
checkpoint_dir = '/content/drive/MyDrive/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# Định nghĩa các callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    os.path.join(checkpoint_dir, 'best_resnet_model.h5'),
    save_best_only=True
)

# Huấn luyện mô hình
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    class_weight=class_weights_dict,
    callbacks=[early_stop, model_checkpoint]
)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.1698 - loss: 2.8694 

470/470 ━━━━━━━━━━━━━━━━━━━━ 6635s 14s/step - accuracy: 0.1699 - loss: 2.8690 - val_accuracy: 0.3256 - val_loss: 1.9582
Epoch 2/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step - accuracy: 0.3054 - loss: 2.3178

470/470 ━━━━━━━━━━━━━━━━━━━━ 186s 395ms/step - accuracy: 0.3054 - loss: 2.3176 - val_accuracy: 0.4767 - val_loss: 1.6598
Epoch 3/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step - accuracy: 0.3739 - loss: 2.1175

...

470/470 ━━━━━━━━━━━━━━━━━━━━ 198s 382ms/step - accuracy: 0.7441 - loss: 0.7858 - val_accuracy: 0.7650 - val_loss: 0.7002
Epoch 41/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 180s 381ms/step - accuracy: 0.7488 - loss: 0.7731 - val_accuracy: 0.7617 - val_loss: 0.7024
Epoch 42/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step - accuracy: 0.7486 - loss: 0.7746

470/470 ━━━━━━━━━━━━━━━━━━━━ 179s 380ms/step - accuracy: 0.7486 - loss: 0.7746 - val_accuracy: 0.7597 - val_loss: 0.6917
Epoch 43/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 178s 380ms/step - accuracy: 0.7547 - loss: 0.7493 - val_accuracy: 0.7557 - val_loss: 0.6927
Epoch 44/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step - accuracy: 0.7514 - loss: 0.7559

470/470 ━━━━━━━━━━━━━━━━━━━━ 181s 385ms/step - accuracy: 0.7514 - loss: 0.7559 - val_accuracy: 0.7676 - val_loss: 0.6773
Epoch 45/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step - accuracy: 0.7572 - loss: 0.7180

470/470 ━━━━━━━━━━━━━━━━━━━━ 205s 391ms/step - accuracy: 0.7572 - loss: 0.7180 - val_accuracy: 0.7690 - val_loss: 0.6753
Epoch 46/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step - accuracy: 0.7605 - loss: 0.7188

470/470 ━━━━━━━━━━━━━━━━━━━━ 181s 385ms/step - accuracy: 0.7605 - loss: 0.7188 - val_accuracy: 0.7670 - val_loss: 0.6720
Epoch 47/50
373/470 ━━━━━━━━━━━━━━━━━━━━ 33s 348ms/step - accuracy: 0.7598 - loss: 0.7160

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam

# Biên dịch mô hình
optimizer = Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Mount Google Drive (nếu chưa mount)
from google.colab import drive
drive.mount('/content/drive')

# Tạo thư mục nếu chưa tồn tại
import os
checkpoint_dir = '/content/drive/MyDrive/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# Định nghĩa các callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    os.path.join(checkpoint_dir, 'best_resnet_model.h5'),
    save_best_only=True
)

# Huấn luyện mô hình
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    class_weight=class_weights_dict,
    callbacks=[early_stop, model_checkpoint]
)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.7570 - loss: 0.7573

470/470 ━━━━━━━━━━━━━━━━━━━━ 5686s 12s/step - accuracy: 0.7571 - loss: 0.7572 - val_accuracy: 0.7643 - val_loss: 0.6720
Epoch 2/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7615 - loss: 0.7254

470/470 ━━━━━━━━━━━━━━━━━━━━ 1752s 4s/step - accuracy: 0.7615 - loss: 0.7253 - val_accuracy: 0.7656 - val_loss: 0.6564
Epoch 3/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7582 - loss: 0.7309

470/470 ━━━━━━━━━━━━━━━━━━━━ 1666s 4s/step - accuracy: 0.7582 - loss: 0.7309 - val_accuracy: 0.7656 - val_loss: 0.6533
Epoch 4/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7715 - loss: 0.6863

470/470 ━━━━━━━━━━━━━━━━━━━━ 1678s 4s/step - accuracy: 0.7715 - loss: 0.6863 - val_accuracy: 0.7703 - val_loss: 0.6493
Epoch 5/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 1684s 4s/step - accuracy: 0.7687 - loss: 0.6870 - val_accuracy: 0.7656 - val_loss: 0.6563
Epoch 6/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7621 - loss: 0.6933

470/470 ━━━━━━━━━━━━━━━━━━━━ 1657s 4s/step - accuracy: 0.7621 - loss: 0.6933 - val_accuracy: 0.7736 - val_loss: 0.6427
Epoch 7/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7550 - loss: 0.7053

KeyboardInterrupt: 

In [ ]:
# Đánh giá mô hình
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Loss trên tập test: {test_loss}')
print(f'Độ chính xác trên tập test: {test_accuracy}')


63/63 ━━━━━━━━━━━━━━━━━━━━ 678s 11s/step - accuracy: 0.7629 - loss: 0.6916
Loss trên tập test: 0.6990572214126587
Độ chính xác trên tập test: 0.7594810128211975
